In [11]:
!jupyter nbconvert --to script --no-prompt starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 794 bytes to starter.py


In [1]:
import pickle
import pandas as pd

In [2]:
with open('./model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [6]:
print(f"The mean trip duration: {y_pred.mean()}")

16.191691679979066

In [7]:
year = 2021
month = 2

In [8]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['prediction']=y_pred
df_results = df[['ride_id','prediction']]

In [9]:
output_file='../output/df_result.parquet'
df_results.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)